<a href="https://colab.research.google.com/github/sitivio/vio/blob/main/TF-IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip3 install -U scikit-learn

In [34]:
import pandas as pd
import numpy as np

data = pd.read_csv("data_preprocessing.csv", usecols=["comment_stemmed"])
data.columns = ["comment_stemmed"]

data.head()

,comment_stemmed
0,"['update', 'tonton', 'video', 'iklan', 'muncul..."
1,"['tolong', 'baik', 'bug', 'nya', 'kesel', 'bng..."
2,"['super', 'temu', 'konten', 'sesuai', 'minat',..."
3,"['kecewa', 'ahh', 'iklan', 'tik', 'tok', 'ga',..."
4,"['iklan', 'sih', 'gapapa', '', 'detik', 'doang..."


In [36]:
#convert list formated string to list
import ast

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]

data["comment_list"] = data["comment_stemmed"].apply(convert_text_list)

print(data["comment_list"][90])

print("\ntype : ", type(data["comment_list"][90]))

['tik', 'tok', 'tik', 'tok', 'life', 'tuh', 'suka', 'mohon', 'tiktok', 'lab', 'permanen', 'nge-like', 'berat', 'buka', 'logo', 'tiktok', 'mohon', 'tikfinity', 'nya', 'fitur', 'handphone', 'tik', 'tok', 'nya', 'langsung', 'aplikasi', 'aplikasi', 'ting', 'nya', 'gampang']

type :  <class 'list'>


In [37]:
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

data["TF_dict"] = data["comment_list"].apply(calc_TF)

data["TF_dict"].head()

0    {'update': 0.045454545454545456, 'tonton': 0.0...
1    {'tolong': 0.058823529411764705, 'baik': 0.088...
2    {'super': 0.034482758620689655, 'temu': 0.0344...
3    {'kecewa': 0.034482758620689655, 'ahh': 0.0344...
4    {'iklan': 0.09090909090909091, 'sih': 0.045454...
Name: TF_dict, dtype: object

In [38]:
# Check TF result
index = 90

print('%20s' % "term", "\t", "TF\n")
for key in data["TF_dict"][index]:
    print('%20s' % key, "\t", data["TF_dict"][index][key])

                term 	 TF

                 tik 	 0.1
                 tok 	 0.1
                life 	 0.03333333333333333
                 tuh 	 0.03333333333333333
                suka 	 0.03333333333333333
               mohon 	 0.06666666666666667
              tiktok 	 0.06666666666666667
                 lab 	 0.03333333333333333
            permanen 	 0.03333333333333333
            nge-like 	 0.03333333333333333
               berat 	 0.03333333333333333
                buka 	 0.03333333333333333
                logo 	 0.03333333333333333
           tikfinity 	 0.03333333333333333
                 nya 	 0.1
               fitur 	 0.03333333333333333
           handphone 	 0.03333333333333333
            langsung 	 0.03333333333333333
            aplikasi 	 0.06666666666666667
                ting 	 0.03333333333333333
             gampang 	 0.03333333333333333


In [39]:
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(data["TF_dict"])

In [40]:
n_document = len(data)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict

#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)

In [41]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
data["TF-IDF_dict"] = data["TF_dict"].apply(calc_TF_IDF)

In [42]:
# Check TF-IDF result
index = 90

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in data["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", data["TF_dict"][index][key] ,"\t" , data["TF-IDF_dict"][index][key])


                term 	         TF 	              TF-IDF

                 tik 	 0.1 	 0.21848020573376623
                 tok 	 0.1 	 0.2199727707554338
                life 	 0.03333333333333333 	 0.21323098850720487
                 tuh 	 0.03333333333333333 	 0.15083758261048516
                suka 	 0.03333333333333333 	 0.0824985439644944
               mohon 	 0.06666666666666667 	 0.16633046570943347
              tiktok 	 0.06666666666666667 	 0.05009885050486058
                 lab 	 0.03333333333333333 	 0.21323098850720487
            permanen 	 0.03333333333333333 	 0.1997154849035994
            nge-like 	 0.03333333333333333 	 0.21323098850720487
               berat 	 0.03333333333333333 	 0.09471938645755776
                buka 	 0.03333333333333333 	 0.0386829230795399
                logo 	 0.03333333333333333 	 0.06494710930911439
           tikfinity 	 0.03333333333333333 	 0.21323098850720487
                 nya 	 0.1 	 0.12759463039510252
               fitur

In [43]:
# sort descending by value for DF dictionary
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

data["TF_IDF_Vec"] = data["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(data["TF_IDF_Vec"][0])

print("\nmatrix size : ", len(data["TF_IDF_Vec"][0]))

print first row matrix TF_IDF_Vec Series

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0748935966024681, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19275123056364465, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21641333040579613, 0.0, 0.0, 0.0, 0.1120501828405373, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

matrix size :  50


In [44]:
# Convert Series to List
TF_IDF_Vec_List = np.array(data["TF_IDF_Vec"].to_list())

# Sum element vector in axis=0
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))

ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)

,term,rank
7,bagus,68.320413
8,,47.675930
0,tiktok,44.133486
2,aplikasi,43.124488
1,buka,39.131566
3,nya,34.629095
6,update,28.566011
13,tik,27.908660
5,baik,27.326887
14,tok,27.008890


In [25]:
data.to_csv('hasil_tfidf.csv')